In [92]:
import difflib
import json
import logging

import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook

from nltk import ngrams

In [23]:
# init logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# init tqdm
try:
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        tqdm_notebook().pandas()
    else:
        tqdm.pandas()
except NameError:
    tqdm.pandas()

#### Load raw data

In [136]:
df_train = pd.read_csv('data/train_data.csv')
df_test = pd.read_csv('data/test_data.csv')
df_train_labels = pd.read_csv('data/train_labels.csv')

In [137]:
df_train = df_train.drop('is_duplicate', axis = 1)
df_train = df_train.merge(df_train_labels)

#Drop rows with missing Qs
df_train = df_train.dropna(axis=0, how='any')


#### Checklist:
- Last Character (Question more likely to be duplicate if they end with same punctuation mark
- Average Shared Words - Words shared by question pairs (more words shared = higher chance of duplicate)
- Shared Entities (Enzo-engineered entities using spaCy)
- Length (word count) - Words per sentence (Questions with different lengths they are unlikely to be duplicates) 
- Levenshtein Features - How many characters difference between two questions (more functions required = more dissimilar)
- Tf-Idf 
- LDA (topic modelling)


https://www.linkedin.com/pulse/kaggle-quora-question-pairs-mar-2017-may-priscilla-li/

## Convenience Method
Rerun below cell, if you make an adjustment to any of the feature methods. Look at examples below for implementation

##### TODO:
- parameter for dataframe or numpy representation
- add padding for word2vec vectors

In [152]:
FEATURE_FUNCTIONS = {
    'last_char': get_last_char,
    'avg_shared_words': get_shared_words,
    'shared_word_percent': shared_word_pcnt,
    'shared_bigrams': get_shared_bigrams,
    'shared_bigrams_percent': shared_bigram_pcnt,
    'shared_trigrams': get_shared_trigrams,
    'shared_trigrams_percent': shared_trigram_pcnt,
    'shared_quadgrams': get_shared_quadgrams,
    'shared_quadgrams_percent': shared_quadgram_pcnt,
    'shared_entities': get_shared_entities,
    'word_count_diff': get_word_count_diff,
    'levenshtein': get_levenshtein_distance,
}

def get_features(df, feature_list):
    """
    Convenience method to extract text features in the same way independent of the 
    dataframe. Ideally this method will be called with a list of feature names as
    a list, which will also work as column names to limit any dataframe to the relevant
    features.
    
    Hint: As pandas dataframes are merely pointers, we can get away with not returning
          the manipulated dataframe as every instance of df will already be affected.
    
    Parameters
        df: dataframe the features will be extracted from
        feature_list: list object, containing all features as strings
        
    Returns
        none: dataframe object is already manipulated and is not needed to be passed back
    """
    for feature in feature_list:
        logging.info('getting {}'.format(feature))
        FEATURE_FUNCTIONS[feature](df)
    logging.info('feature extraction done')

In [153]:
# === Example 1 ===
get_features(df_test, ['last_char', 'avg_shared_words', 'word_count_diff', 'shared_entities', 'levenshtein', 'shared_word_percent'])
df_test.head()

INFO:root:getting shared_quadgrams


INFO:root:getting shared_quadgrams_percent


INFO:root:feature extraction done


,test_id,question1,question2,avg_shared_bigrams,shared_bigram_pcnt,avg_shared_trigrams,shared_trigram_pcnt,avg_shared_quadgrams,shared_quadgram_pcnt
0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,2,0.133333,1,0.071429,0,0.0
1,20,Why do rockets look white?,Why are rockets and boosters painted white?,0,0.000000,0,0.000000,0,0.0
2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,0,0.000000,0,0.000000,0,0.0
3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,0,0.000000,0,0.000000,0,0.0
4,34,What is the best travel website in spain?,What is the best travel website?,4,0.666667,3,0.600000,2,0.5


In [142]:
# === Example 2 ===
df_ents = pd.read_csv('data/test_with_sim_and_ents.csv')
get_features(df_ents, ['shared_entities'])
df_ents.head()

INFO:root:getting shared_entities
INFO:root:converting entity 1 string representation to dictionaries


HBox(children=(IntProgress(value=0, max=81126), HTML(value=u'')))

INFO:root:converting entity 2 string representation to dictionaries


HBox(children=(IntProgress(value=0, max=81126), HTML(value=u'')))

INFO:root:looking up shared entities


HBox(children=(IntProgress(value=0, max=81126), HTML(value=u'')))

INFO:root:feature extraction done


,Unnamed: 0,test_id,question1,question2,entities1,entities2,similarity_score,shared_entities
0,0,15,invalid,How will a Trump presidency affect the student...,None,"{u'ORG': 1, u'GPE': 2}",0.235347,0
1,1,20,Why do rockets look white?,Why are rockets and boosters painted white?,None,None,0.892270,0
2,2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,None,None,0.956441,0
3,3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,"{u'PRODUCT': 1, u'QUANTITY': 1}",{u'ORG': 1},0.835993,0
4,4,34,What is the best travel website in spain?,What is the best travel website?,{u'GPE': 1},None,0.968766,0


## Features

### Last Character

In [9]:
def get_last_char(df):
    df['last_char'] = np.where(df.question1.str[-1:] == df.question2.str[-1:], 1, 0)

### Avg. Shared Words

In [89]:
def shared_word_count(q1, q2):
    return len([w for w in lower_list(q1.split(' ')) if w in lower_list(q2.split(' '))])

def get_shared_words(df):
    df['avg_shared_words'] = df.progress_apply(lambda row: shared_word_count(row['question1'],                                                                         
                                                                             row['question2']), axis=1)
    
#Shared word percentage (0-100%)   
def avg_word_count(q1, q2):
    return (len(lower_list(q1.split(' '))) + len(lower_list(q2.split(' '))))/2.0
    
def shared_word_pcnt(df):
    df['shared_words_pcnt'] = df.progress_apply(lambda row: shared_word_count(row['question1'], 
                                                                             row['question2'])/ avg_word_count(row['question1'], 
                                                                             row['question2']), axis=1)    
    

### Shared ngrams

In [144]:
def shared_bigram_count(q1, q2):
    return len([w for w in ngrams(lower_list(q1.split(' ')),2) if w in ngrams(lower_list(q2.split(' ')),2)])

def get_shared_bigrams(df):
    df['avg_shared_bigrams'] = df.progress_apply(lambda row: shared_bigram_count(row['question1'],row['question2']), axis=1)

   
## percentage

def bigram_count(q1, q2):
    cnt = ((len([x for x in ngrams(lower_list(q1.split(' ')),2)]) + len([x for x in ngrams(lower_list(q2.split(' ')),2)]))/2.0)
    if cnt >0:
        return cnt
    else:
        return 1
            
def shared_bigram_pcnt(df):
    df['shared_bigram_pcnt'] = df.progress_apply(lambda row: shared_bigram_count(row['question1'], 
                                                                             row['question2'])/ bigram_count(row['question1'], 
                                                                             row['question2']), axis=1) 

### Trigrams

In [148]:
def shared_trigram_count(q1, q2):
    return len([w for w in ngrams(lower_list(q1.split(' ')),3) if w in ngrams(lower_list(q2.split(' ')),3)])

def get_shared_trigrams(df):
    df['avg_shared_trigrams'] = df.progress_apply(lambda row: shared_trigram_count(row['question1'],row['question2']), axis=1)

   
## percentage

def trigram_count(q1, q2):
    cnt = ((len([x for x in ngrams(lower_list(q1.split(' ')),3)]) + len([x for x in ngrams(lower_list(q2.split(' ')),3)]))/2.0)
    if cnt >0:
        return cnt
    else:
        return 1
            
def shared_trigram_pcnt(df):
    df['shared_trigram_pcnt'] = df.progress_apply(lambda row: shared_trigram_count(row['question1'], 
                                                                             row['question2'])/ trigram_count(row['question1'], 
                                                                             row['question2']), axis=1) 

### Quadgrams?!

In [151]:
def shared_quadgram_count(q1, q2):
    return len([w for w in ngrams(lower_list(q1.split(' ')),4) if w in ngrams(lower_list(q2.split(' ')),4)])

def get_shared_quadgrams(df):
    df['avg_shared_quadgrams'] = df.progress_apply(lambda row: shared_quadgram_count(row['question1'],row['question2']), axis=1)

   
## percentage

def quadgram_count(q1, q2):
    cnt = ((len([x for x in ngrams(lower_list(q1.split(' ')),4)]) + len([x for x in ngrams(lower_list(q2.split(' ')),4)]))/2.0)
    if cnt >0:
        return cnt
    else:
        return 1
            
def shared_quadgram_pcnt(df):
    df['shared_quadgram_pcnt'] = df.progress_apply(lambda row: shared_quadgram_count(row['question1'], 
                                                                             row['question2'])/ quadgram_count(row['question1'], 
                                                                             row['question2']), axis=1) 

### Shared Entities

In [11]:
ENTITIES = {u'CARDINAL', u'DATE', u'EVENT', u'FAC', u'GPE', u'LANGUAGE', u'LAW', u'LOC', u'MONEY', u'NORP', u'ORDINAL',
            u'ORG', u'PERCENT', u'PERSON', u'PRODUCT', u'QUANTITY', u'TIME', u'WORK_OF_ART'}

In [12]:
def shared_keys(d1, d2):
    d1 = {} if not isinstance(d1, dict) else d1
    d2 = {} if not isinstance(d2, dict) else d2
    return len([w for w in d1.keys() if w in d2.keys()])

def get_shared_entities(df):
    if 'entities1' not in df.columns or 'entities2' not in df.columns:
        logging.warning('No entity dicts found in provided dataframe')
        return
    if isinstance(df.entities1[df.entities1.first_valid_index()], str):
        logging.info('converting entity 1 string representation to dictionaries')
        df.entities1 = df.entities1.progress_apply(lambda x: string_to_dict(x))
    if isinstance(df.entities2[df.entities2.first_valid_index()], str):
        logging.info('converting entity 2 string representation to dictionaries')
        df.entities2 = df.entities2.progress_apply(lambda x: string_to_dict(x))
    
    logging.info('looking up shared entities')
    df['shared_entities'] = df.progress_apply(lambda row: shared_keys(row['entities1'], 
                                                                      row['entities2']), axis=1)

### Word Count Diff

In [13]:
def count_diff(q1, q2):
    return abs(len(q1.split(' ')) - len(q2.split(' ')))

def get_word_count_diff(df):
    df['word_count_diff'] = df.progress_apply(lambda row: count_diff(row['question1'], 
                                                                     row['question2']), axis=1)

### Levenshtein Distance

In [14]:
def get_levenshtein_distance(df):
    # TODO: discuss if spaces should be considered in the distance calculation, add 'lambda x: x == " "' as first parameter to exclude spaces
    df['levenshtein'] = df.progress_apply(lambda row: difflib.SequenceMatcher(None, 
                                                                              row['question1'], 
                                                                              row['question2']).ratio(), axis=1)

## Helpers

In [27]:
def lower_list(_list):
    return [x.lower() for x in _list]

In [28]:
def string_to_dict(dict_string):
    if isinstance(dict_string, str):
        # Convert to proper json format
        dict_string = dict_string.replace("'", '"').replace('u"', '"')
        return json.loads(dict_string)

## Export

In [81]:
#train_data
get_features(df_train, ['last_char', 'avg_shared_words', 'word_count_diff', 'shared_entities', 'levenshtein'])
df_train.to_csv("train_features.csv", index=False)
#test_data
get_features(df_test, ['last_char', 'avg_shared_words', 'word_count_diff', 'shared_entities', 'levenshtein'])
df_test.to_csv("test_features.csv", index=False)

INFO:root:getting last_char
INFO:root:getting avg_shared_words


INFO:root:getting word_count_diff


INFO:root:getting shared_entities
INFO:root:getting levenshtein


INFO:root:feature extraction done
INFO:root:getting last_char
INFO:root:getting avg_shared_words


INFO:root:getting word_count_diff


INFO:root:getting shared_entities
INFO:root:getting levenshtein


INFO:root:feature extraction done
